# Deep Q-Network

# Import

In [1]:
import gym
import random
import numpy as np
from collections import deque
from time import sleep
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# Environment

In [2]:
env = gym.make('CartPole-v0').env

In [4]:
env.observation_space

Box(4,)

In [5]:
env.action_space

Discrete(2)

# Model

In [6]:
class DQN(Sequential):
    def __init__(self, n_nodes=32, n_hidden=2):
        super().__init__()
        # input layer
        self.add(Dense(n_nodes, activation='relu', input_shape=env.observation_space.shape), )
        # hidden layers
        for _ in range(n_hidden):
            self.add(Dense(n_nodes, activation='relu'), )
        # output layer
        self.add(Dense(env.action_space.n, activation='linear'),)
        # compile
        self.compile(loss='mse', optimizer='adam')

    def clone_from(self, another):
        self.set_weights(another.get_weights())
        return self

In [7]:
policy = DQN()

# Replay Memory

In [8]:
class Memory:
    def __init__(self, maxlen):
        self._memory = deque(maxlen=maxlen)
    
    def remember(self, state, action, reward, next_state, done):
        self._memory.append((state, action, reward, next_state, done))
    
    def get_batch(self, batch_size):
        samples = random.sample(self._memory, min(len(self._memory), batch_size))
        batch = np.array(samples, dtype=object).transpose()
        states, actions, rewards, next_states, dones = batch
        states, next_states = np.stack(states), np.stack(next_states)
        return states, actions, rewards, next_states, dones

# Agent

In [9]:
class Agent:
    def __init__(self, env, policy):
        self._env = env
        self._memory = Memory(100000)
        self._policy = policy
        self._target = DQN().clone_from(self._policy)
    
    def _choose_action(self, state, *, epilson=0.5):
        if np.random.random()>epilson:
            return self._env.action_space.sample()
        else:
            return np.argmax(agent._policy(tf.constant([state])))       

In [10]:
class Agent(Agent):
    def play(self, *, n_steps=200, render=False):
        state = env.reset()
        done = False
        rewards = 0
        for i_steps in range(1, n_steps+1):
            action = self._choose_action(state, epilson=1)
            next_state, reward, done, info = env.step(action)
            if render: env.render()
            rewards += reward
            state = next_state
            if done: break
        if render: 
            print(f'Steps taken: {i_steps}, rewards earned: {rewards}')
            env.close()
        else:
            return rewards

In [11]:
class Agent(Agent):
    def _train(self, *, batch_size=512, gamma=0.99):
        states, actions, rewards, next_states, dones = self._memory.get_batch(batch_size)
        next_states_predicts = self._target(next_states).numpy()
        q_targets = self._policy(states).numpy()
        for i,row in enumerate(q_targets):
            row[actions[i]] = rewards[i] if dones[i] else rewards[i] + gamma*np.max(next_states_predicts[i])
        X, y = tf.constant(states), tf.constant(q_targets)
        self._policy.fit(X, y, epochs=10, batch_size=len(X), verbose=0)

In [12]:
class Agent(Agent):
    def run(self, *, n_eps=3000, n_steps=200):
        scores = deque(maxlen=10)
        for i_eps in range(1, n_eps+1):            
            state = env.reset()
            done = False            
            for _ in range(n_steps):
                action = self._choose_action(state, epilson=i_eps/n_eps)
                next_state, reward, done, info = env.step(action)
                self._memory.remember(state, action, reward, next_state, done)
                state = next_state 
                if done: break
            self._train()
            if i_eps%10==0:
                self._target.clone_from(self._policy)
                scores.append(self.play())
            if i_eps%2==0:
                print('#', end='')
            if i_eps%100==0:            
                mean_score = sum(scores)/len(scores)
                print(f' | Episode {i_eps:>4d} | rewards: {mean_score:.1f}')
                if mean_score>=195:
                    print(f'\nMean score of {mean_score:.1f} has reached the target.')
                    break

In [13]:
agent = Agent(env, policy)

In [14]:
agent.run()

################################################## | Episode  100 | rewards: 79.3
################################################## | Episode  200 | rewards: 187.4
################################################## | Episode  300 | rewards: 200.0

Mean score of 200.0 has reached the target.


# Evaluation

In [17]:
agent.play(render=True)

Steps taken: 200, rewards earned: 200.0


In [16]:
if input('Save model? ([Y]/n)').upper()=='Y':
    policy.save('cartpole_v0@DQN#keras.h5')

Save model? ([Y]/n) y


# Comment

* This environment has continuous state space but discrete action space, making simple Q-Learning not suitable.
* Using DQN to handle this environment is fairly easily, the agent can almost solve the game in early stage.
* Use of epsilon greedy strategy allows early exploration, and those experience will continue to be learned by using a larger replay memory size.
* Too many episode runs may not be a good idea as you can see in later stage the model rewards starts to fluctuate and may be a indication of overfit. Therefore may consider applying early-stopping strategy.